<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 4: Aproximación de funciones

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
from anmi.genericas import norma_p_func, norma_inf_func

from anmi.T4 import metodo_ruffini
from anmi.T4 import producto_deriv, producto_asecas, producto_escalar_peso, metodo_gram, gram_schmidt_f, polinomios_orto_peso
from anmi.T4 import producto_escalar_trigono, coefs_fourier, coefs_fourier_discr, serie_fourier
from anmi.T4 import polinomios_bernstein

In [ ]:
x, y, z, a, lambda_ = symbols('x'), symbols('y'), symbols('z'), symbols('a'), symbols('lambda')

El objetivo de la aproximación de funciones es encontrar una función de caracteristicas menores o con términos más fáciles para computar, que se aproxime a una función dada en un intervalo o conjunto de puntos determinado. A nivel computacional, también es más eficiente y estable poder emplear cierto tipo de funciones de aproximación.

## Representación anidada (algoritmo de Horner) / Método de Ruffini
Si tenemos un polinomio $p(x) = a_0 + a_1x + a_2x^2+\cdots+a_nx^n$, entonces podemos hacer el polinomio más compacto, en la forma $p(x) = q_1(q_2(\cdots(q_n(x)))$, tal que $q_i(x) = a_{i-1} + a_i(x)$.

In [ ]:
help(metodo_ruffini)

In [ ]:
poli_1 = Poly(x**3 - x**2 + 2*x - 5)

In [ ]:
metodo_ruffini(poli_1, 1)

## Aproximación de funciones
Si $f$ es una función en un espacio vectorial $V$ podemos definir una norma $||\;\;||$, que es una aplicación que cumple:
* $||f|| = 0  \iff f = 0$
* $||\lambda f|| = |\lambda|\,||f||$
* $||f + g|| \le ||f|| + ||g||$

Si fijamos un subespacio $U \subset V$, podemos buscar una función $g\in U$ tal que sea la más similar.

Si queremos medir la distancia entre 2 funciones, hay múltiples opciones:
* Norma 2: $||f||_2 = \left( \int_a^bf(x)^2 dx \right)^\frac{1}{2}$
* Norma p: $||f||_p = \left( \int_a^b |f(x)|^p dx \right)^\frac{1}{p}$
* Norma inf: $||f||_{\inf} = \max_{x \in [a,b]} |f(x)|$

In [ ]:
f = x
g1 = sin(x)
g2 = 1 - cos(x)

In [ ]:
norma_p_func(f - g1, p=2, a=0, b=2*pi)

In [ ]:
norma_inf_func(f - g1, a=0, b=2*pi)

In [ ]:
norma_p_func(f - g2, p=2, a=0, b=2*pi)

In [ ]:
norma_inf_func(f - g2, a=0, b=2*pi)

### Aproximación por mínimos cuadrados (continua y discreta)

Para la aproximación definimos el producto escalar como una aplicación $\langle\,,\rangle \; :\,V \times V \rightarrow \mathbb{R}^+$ que verifica:
* $\langle f, f\rangle = \iff f = 0$
* $\langle f, g\rangle =\langle g, f\rangle$
* $\langle\alpha f+\mu g, h\rangle = \alpha\langle f, h\rangle + \mu \langle g, h\rangle$

La norma se define como $||f|| = \langle f, f\rangle^{\frac{1}{2}}$.

Una forma de producto escalar es, por ejemplo:
$$\langle f, g \rangle = \int_a^b f(x)g(x) dx$$

Esa forma puede aplicarse de forma discreta (para un conjunto discreto $I = \{x_1, x_2, \cdots, x_m\}$ como :
$$\langle f, g \rangle = \sum_{i \in I} f(i)g(i)$$ 

En la aproximación de mínimos cuadrados deseamos encontrar, para una base $\{\phi_0, \cdots, \phi_n\}$ de $U$, un vector $\alpha$ que minimice una función de coste $J(\alpha) = ||f - \sum_0^n \alpha_i\phi_i||^2$. Para ello se emplea el método de Gram, que consiste en encontrar la solución a 
$$G\alpha = \bar{f}$$
donde $G_{ij} = \langle\phi_i, \phi_j\rangle$ y $\bar{f}_i = \langle f, \phi_i\rangle$ en un producto escalar definido arbitrariamente.

In [ ]:
help(producto_deriv)
help(producto_asecas)

In [ ]:
help(metodo_gram)

#### Ejercicio 33

In [ ]:
U = [S(1), x, x ** 2]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = E ** x
dict_prod_deriv = metodo_gram(f, U, var=x, func_producto=producto_deriv, a=0, b=1)

In [ ]:
dict_prod_deriv['poly']

In [ ]:
dict_prod_deriv['G']

In [ ]:
dict_prod_deriv['f_bar']

In [ ]:
dict_prod_deriv['alpha']

Ahora con una versión discreta en los puntos 0, 0.2, 0.4, 0.6, 0.8, 1:

In [ ]:
U = [S(1), x, x ** 2]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = E ** x
dict_prod_deriv_discreto = metodo_gram(f, U, var=x, func_producto=producto_deriv, I = [0, 0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
dict_prod_deriv_discreto['poly']

In [ ]:
dict_prod_deriv_discreto['G']

In [ ]:
dict_prod_deriv_discreto['f_bar']

In [ ]:
dict_prod_deriv_discreto['alpha']

Ahora la versión continua pero con la función $\langle f, g \rangle = \int_a^b f(x)g(x) dx$

In [ ]:
U = [S(1), x, x ** 2]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = E ** x
dict_prod_asecas = metodo_gram(f, U, var=x, func_producto=producto_asecas, a=0, b=1)

In [ ]:
dict_prod_asecas['poly']

In [ ]:
x_range = np.linspace(0, 1, 100)
y_real = [(E ** x).subs(x, i) for i in x_range]
y_prod_deriv = [dict_prod_deriv['poly'].subs(x, i) for i in x_range]
y_prod_discreto = [dict_prod_deriv_discreto['poly'].subs(x, i) for i in x_range]
y_prod_asecas = [dict_prod_asecas['poly'].subs(x, i) for i in x_range]

plt.plot(x_range, y_real, label='real')
plt.plot(x_range, y_prod_deriv, label="fg + f'g' continuo")
plt.plot(x_range, y_prod_discreto, label="fg + f'g' discreto")
plt.plot(x_range, y_prod_asecas, label="fg")
plt.legend()

#### Ejercicio 34
En el ejercicio piden usar la base $\{1, x\}$. Nosotros lo hacemos con $\{1, x, x^2, x^3, x^4, x^5\}$, que da una representación más real de la aproximación.

Calculamos la versión continua

In [ ]:
U = [S(1), x, x ** 2, x ** 3, x ** 4]  # S(1) lo hacemos porque hacer 1.diff da error por ser int.
f = cos(pi * x)/2 + sin(pi/2 * x)/3
dict_prod_continuo = metodo_gram(f, U, var=x, func_producto=producto_asecas, a=-1, b=1)

In [ ]:
dict_prod_continuo['poly']

In [ ]:
dict_prod_continuo['G']

In [ ]:
dict_prod_continuo['f_bar']

In [ ]:
dict_prod_continuo['alpha']

Y ahora la discreta

In [ ]:
dict_prod_discreto = metodo_gram(f, U, var=x, func_producto=producto_asecas, I=[-1, 0, 1])

In [ ]:
dict_prod_discreto['poly']

In [ ]:
dict_prod_discreto['G']

In [ ]:
dict_prod_discreto['f_bar']

In [ ]:
dict_prod_discreto['alpha']

Repetimos el proceso, por curiosidad, pero con el producto escalar fg + f'g'

In [ ]:
dict_prod_continuo_deriv = metodo_gram(f, U, var=x, func_producto=producto_deriv, a=-1, b=1)
dict_prod_discreto_deriv = metodo_gram(f, U, var=x, func_producto=producto_deriv, I=[-1, 0, 1])

In [ ]:
dict_prod_continuo_deriv['poly']

In [ ]:
dict_prod_discreto_deriv['poly']

In [ ]:
x_range = np.linspace(-1, 1, 100)
y_real = [f.subs(x, i) for i in x_range]
y_prod_continuo = [dict_prod_continuo['poly'].subs(x, i) for i in x_range]
y_prod_discreto = [dict_prod_discreto['poly'].subs(x, i) for i in x_range]
y_prod_continuo_deriv = [dict_prod_continuo_deriv['poly'].subs(x, i) for i in x_range]
y_prod_discreto_deriv = [dict_prod_discreto_deriv['poly'].subs(x, i) for i in x_range]

plt.plot(x_range, y_real, label='real')
plt.plot(x_range, y_prod_continuo, label="fg continuo")
plt.plot(x_range, y_prod_discreto, label="fg discreto")
plt.plot(x_range, y_prod_continuo_deriv, label="fg + f'g' continuo")
plt.plot(x_range, y_prod_discreto_deriv, label="fg + f'g' discreto")

plt.legend()

Si nos fijamos, la aproximación con el método discreto es bastante mala, al no encontrar una inversa adecuada.

In [ ]:
help(gram_schmidt_f)

#### Ejemplo 23

In [ ]:
gram_schmidt_f([S(1), x, x ** 2, x ** 3, x ** 4, x ** 5, x ** 6], x, prod_esc=producto_asecas)

#### Ejercicio 35

In [ ]:
I = [-1, 0, 1, 2]
U_GS = gram_schmidt_f([S(1), x, x ** 2], x, prod_esc=producto_asecas, I=I)

In [ ]:
for i in U_GS:
    print(i)

In [ ]:
f = sin(pi/2 * x)
dict_prod_discreto = metodo_gram(f, U_GS, var=x, func_producto=producto_asecas, I=I)

In [ ]:
dict_prod_discreto['poly']

### Aproximación con peso

Para las anteriores definiciones podemos tomar un producto escalar
$$\langle f, g \rangle = \int_a^b f(x)g(x)\omega(x) dx$$

Donde $\omega(x)$ es una función positiva en $(a,b)$

Se puede construir la sucesión de polinomios
$$p_0(x) = 1$$
$$p_1(x) = x - a_1$$
$$p_2(x) = (x - a_2)p_1(x) - b_2p_0(x)$$
$$\cdots$$
$$p_n(x) = (x - a_n)p_{n-1}(x) - b_np_{n-2}(x)$$

Con
$$a_n = \frac{\langle xp_{n-1},p_{n-1}\rangle}{\langle p_{n-1},p_{n-1}\rangle}$$
$$b_n = \frac{\langle xp_{n-1},p_{n-2}\rangle}{\langle p_{n-2},p_{n-2}\rangle}$$

In [ ]:
help(polinomios_orto_peso)           

#### Polinomios de Legendre
Los polinomios de Legendre surgen de la aplicación del método descrito anteriormente para la base $\mathcal{P}(n) = \{1, x, x^2, \cdots, x^n\}$.

In [ ]:
# La base para los polinomios tiene que ser ortogonal porque si no los polinomios no son ortogonales entre si!
base = [S(1), x, x**2, x**3, x**4, x**5, x**6]
base_ortogonal = gram_schmidt_f(base, var=x, prod_esc=producto_asecas, a=-1, b=1)
base_legendre = polinomios_orto_peso(base_ortogonal, w=S(1), var=x, a=-1, b=1)

In [ ]:
base_legendre

In [ ]:
x_range = np.linspace(-1, 1, 100)

for p in base_legendre['p']:
    plt.plot(x_range, [p.subs(x, i)/abs(p.subs(x, -1)) for i in x_range], label=S(1)/abs(p.subs(x, -1)))
            
plt.legend(bbox_to_anchor=(1, 0.5))

Ahora probamos a resolver algun problema asociado. Por ejemplo, vamos a aproximar sin(x) en [0, 2 * pi]

In [ ]:
base = [S(1), x, x**2, x**3]
base_ortogonal = gram_schmidt_f(base, var=x, prod_esc=producto_asecas, a=0, b=2*pi)
base_legendre = polinomios_orto_peso(base_ortogonal, w=S(1), var=x, a=0, b=2*pi)

In [ ]:
f = sin(x)

U_GS_base = metodo_gram(f, base, var=x, func_producto=producto_escalar_peso, a=0, b=2*pi, w=S(1))

U_GS_ortogonal = metodo_gram(f, base_ortogonal, var=x, func_producto=producto_escalar_peso, a=0, b=2*pi, w=S(1))

U_GS_legendre = metodo_gram(f, base_legendre['p'], var=x, func_producto=producto_escalar_peso, a=0, b=2*pi, w=S(1))

In [ ]:
expand(U_GS_base['poly'])

In [ ]:
expand(U_GS_ortogonal['poly'])

In [ ]:
expand(U_GS_legendre['poly'])

Los polinomios son técnicamente los mismos. Sin embargo, las matrices de Gram son mucho más convenientes para la base ortogonal / de legendre.

In [ ]:
U_GS_base['G']

In [ ]:
U_GS_ortogonal['G']

In [ ]:
U_GS_legendre['G']

Ahora vamos a hacer la aproximación gráfica para las funciones $\sin(x)$ y $\ln(x)$

In [ ]:
x_range = np.linspace(0, 2 * np.pi, 100)

base = [S(1), x, x**2, x**3, x**4, x**5, x**6, x**7]
f = sin(x)

plt.plot(x_range, [sin(x).subs(x,i) for i in x_range], label='real')
for i in range(3, 8, 2):
    base_sub = base[:i]
    U_GS_base = metodo_gram(f, base_sub, var=x, func_producto=producto_escalar_peso, a=0, b=2*pi, w=S(1))
    plt.plot(x_range, [U_GS_base['poly'].subs(x,i) for i in x_range], label=f'poly_{i}')
            
plt.legend(bbox_to_anchor=(1, 0.5))

In [ ]:
x_range = np.linspace(0.01, 2, 100)

base = [S(1), x, x**2, x**3, x**4, x**5, x**6, x**7,]
f = ln(x)/ln(2)

plt.plot(x_range, [f.subs(x,i) for i in x_range], label='real')
for i in range(3, 8, 2):
    base_sub = base[:i]
    U_GS_base = metodo_gram(f, base_sub, var=x, func_producto=producto_escalar_peso, a=0.01, b=2, w=S(1))
    plt.plot(x_range, [U_GS_base['poly'].subs(x,i) for i in x_range], label=f'poly_{i}')
            
plt.legend(bbox_to_anchor=(1.5, 0.3))

Vemos que una aproximación polinomial de log es bastante buena también!

### Polinomios de Chebyshev
Los polinomios de Chevysheb responden a la ecuación $T_n = \cos(n \arccos(x))$. En base a las propiedades trigonométricas, estos polinomios también responden a la forma:
$$T_0 = 1, \;\; T_1 = x$$
$$T_n = 2xT_{n-1} - T_{n-2}$$
En el intervalo [-1, 1]. Fuera de él la expresión T_n da valores complejos, aunque la parte real de esos valores se aproxima mucho a la forma polinomial.

Estos polinomios son ortogonales en el intervalo [-1, 1] estableciendo $\omega(x) = \frac{1}{\sqrt{1-x^2}}$:
$$\int_{-1}^{1} T_nT_m \frac{1}{\sqrt{1-x^2}} dx = \begin{cases}
0 & n\neq m \\
\pi & n = m = 0 \\
\pi/2 & n = m \neq 0
\end {cases}$$

In [ ]:
# Aqui los generamos iterativamente
t0 = S(1)
t1 = x
t2 = expand(2 * x * t1 - t0)
t3 = expand(2 * x * t2 - t1)
t4 = expand(2 * x * t3 - t2)
t5 = expand(2 * x * t4 - t3)
t5

In [ ]:
# También podemos usar la función de sympy
chebyshevt_poly(5)

In [ ]:
T5 = cos(5 * acos(x))

In [ ]:
x_range = np.linspace(-3, 3, 1000)
y_t5 = [t5.subs(x, i) for i in x_range]
y_T5 = [re(T5.subs(x, i)) for i in x_range]

plt.plot(x_range, y_t5, label='t5')
plt.plot(x_range, y_T5, label="T5")
plt.legend()

Vemos que ambos polinomios son iguales

Ahora vamos a comprobar la ortogonalidad de los polinomios usando el peso adecuado, u otro peso

In [ ]:
producto_escalar_peso(chebyshevt_poly(3, x), chebyshevt_poly(5, x), x, w=S(1),a=-1, b=1)

In [ ]:
producto_escalar_peso(chebyshevt_poly(3, x), chebyshevt_poly(5, x), x, w=1/sqrt(1 - x**2),a=-1, b=1)

Vemos que si no añadimos el peso $\omega(x)$ el resultado de la integral no es 0.

In [ ]:
producto_escalar_peso(chebyshevt_poly(0, x), chebyshevt_poly(0, x), x, w=1/sqrt(1 - x**2),a=-1, b=1)

In [ ]:
producto_escalar_peso(chebyshevt_poly(1, x), chebyshevt_poly(1, x), x, w=1/sqrt(1 - x**2),a=-1, b=1)

In [ ]:
producto_escalar_peso(chebyshevt_poly(3, x), chebyshevt_poly(3, x), x, w=1/sqrt(1 - x**2),a=-1, b=1)

In [ ]:
producto_escalar_peso(chebyshevt_poly(5, x), chebyshevt_poly(5, x), x, w=1/sqrt(1 - x**2),a=-1, b=1)

Para dos polinomios iguales, los resultados son los esperados

### Aproximación trigonométrica (series de Fourier)
La aproximación trigonométrica considera el siguiente producto escalar:
$$\langle f, g\rangle = \frac{1}{2\pi}\int_{-\pi}^{\pi}f(x)g(x)dx$$

Si en el método de Gram consideramos los polinomios trigonométricos $\{1, \cos(x), \sin(x), \cdots, \cos(nx), \sin(nx)\}$ tenemos que $\langle f, g\rangle$ para la base $\mathcal{U}$ es:
$$\int_{-\pi}^{\pi} \cos(kx)\sin(jx)dx = 0$$
$$\int_{-\pi}^{\pi} \cos(kx)\cos(jx)dx = \int_{-\pi}^{\pi} \cos(kx)\cos(jx)dx = 
\begin{cases}0 & k\neq j \\ \pi & k=j>1 \\ 2\pi & k = j = 0\end{cases}$$

Así, la matriz a resolver es:
$$\begin{bmatrix}
2\pi & 0 & 0 & \cdots & 0 & 0 \\
0  & \pi & 0 & \cdots & 0 & 0 \\
0 & 0 & \pi  & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0  & 0 & 0 & \cdots & \pi & 0 \\
0 & 0 & 0  & \cdots & 0 & \pi \\
\end{bmatrix}
\begin{bmatrix}
 a_0  \\
 a_1  \\
 b_1 \\
\vdots \\
 a_n \\
 b_n  \\
\end{bmatrix}
=
\begin{bmatrix}
\langle f, 1 \rangle \\
\langle f, \cos(x) \rangle \\
\langle f, \sin(x) \rangle \\
\vdots \\
\langle f, \cos(nx) \rangle \\
\langle f, \sin(nx) \rangle \\
\end{bmatrix}
$$

Luego
$$
\begin{bmatrix}
 a_0  \\
 a_1  \\
 b_1 \\
\vdots \\
 a_n \\
 b_n  \\
\end{bmatrix}
=
\begin{bmatrix}
\frac{1}{2\pi} & 0 & 0 & \cdots & 0 & 0 \\
0  & \frac{1}{\pi} & 0 & \cdots & 0 & 0 \\
0 & 0 & \frac{1}{\pi}  & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0  & 0 & 0 & \cdots & \frac{1}{\pi} & 0 \\
0 & 0 & 0  & \cdots & 0 & \frac{1}{\pi} \\
\end{bmatrix}
\begin{bmatrix}
\langle f, 1 \rangle \\
\langle f, \cos(x) \rangle \\
\langle f, \sin(x) \rangle \\
\vdots \\
\langle f, \cos(nx) \rangle \\
\langle f, \sin(nx) \rangle \\
\end{bmatrix} = 
\begin{bmatrix}
\frac{1}{2\pi}\langle f, 1 \rangle \\
\frac{1}{\pi}\langle f, \cos(x) \rangle \\
\frac{1}{\pi}\langle f, \sin(x) \rangle \\
\vdots \\
\frac{1}{\pi}\langle f, \cos(nx) \rangle \\
\frac{1}{\pi}\langle f, \sin(nx) \rangle \\
\end{bmatrix}
$$

Así, los coeficientes quedan definidos como:
$$a_k = \frac{1}{\pi} \int_{-\pi}^{\pi} f(x) \cos(kx) dx$$
$$b_k = \frac{1}{\pi} \int_{-\pi}^{\pi} f(x) \sin(kx) dx$$

Si combinamos los coeficientes con los elementos de $\mathcal{U}$ tenemos que el polinomio óptimo es, hasta grado $n$:
$$g_n(x) = \frac{a_0}{2} + \sum_{k=1}^n a_k\cos(kx) + b_k\sin(kx)$$

In [ ]:
help(producto_escalar_trigono)

#### EJERCICIO 36

In [ ]:
base = [S(1), cos(x), sin(x)]

m_gram = zeros(3, 3)
f_base = zeros(3, 1)

for i in range(3):
    for j in range(3):
        m_gram[i, j] = producto_escalar_trigono(base[i], base[j], x)
    
    f_base[i, 0] = producto_escalar_trigono(x, base[i], x)

In [ ]:
m_gram  # Se ve que la base es ortogonal

In [ ]:
f_base

In [ ]:
# Resolvemos la matriz de Gram
g = ((m_gram ** -1 * f_base).T * mat([base]).T)[0]
g

In [ ]:
help(coefs_fourier)
help(coefs_fourier_discr)
help(serie_fourier)

#### EJEMPLO 25 ($f(x) = e^x$)
En este ejercicio vamos a calcular la expansión de Fourier para órdenes 3, 6 y 15. Comprobamos que en orden 6 la aproximación es mejor, pero en los extremos del intervalo la aproximación empeora, tal y como se comenta en el texto base.

In [ ]:
f = E ** x

In [ ]:
coefs_fourier(f, var=x, I=[-pi, pi], n_coefs=6)

In [ ]:
serie_f_3 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=3))
serie_f_6 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=6))
serie_f_15 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=15))

In [ ]:
x_range = np.linspace(-np.pi, np.pi, 300)
real = [(f).subs(x, i) for i in x_range]
f3 = [serie_f_3.subs(x, i) for i in x_range]
f6 = [serie_f_6.subs(x, i) for i in x_range]
f15 = [serie_f_15.subs(x, i) for i in x_range]

plt.plot(x_range, real, label="E**x")
plt.plot(x_range, f3, label='f3')
plt.plot(x_range, f6, label="f6")
plt.plot(x_range, f15, label="f15")

plt.legend()

Las series de Fourier también admiten un caso discreto. En este caso, si dividimos el intervalo $[a,b]$ en $2m$ (asumimos $m$ hasta $[a, \frac{a+b}{2}]$ y otros $m$ hasta $[\frac{a+b}{2}, b]$), para el caso discreto los coeficientes $a_k$ y $b_k$ se redefinen como:
$$a_k = \frac{1}{m}\sum_{i=0}^{2m-1}f(x_i)\cos(kx_i)$$
$$b_k = \frac{1}{m}\sum_{i=0}^{2m-1}f(x_i)\sin(kx_i)$$

que, por simplificar, sería una media de $m$ valores en el intervalo, en lugar de la integral. Cuanto más grande sea $m$, más se van a parecer los coeficientes del caso discreto al continuo.

Ahora hacemos el caso discreto para m=6

In [ ]:
serie_f_6_m_3 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=3))

In [ ]:
serie_f_6_m_10 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=10))

In [ ]:
serie_f_6_m_20 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=20))

In [ ]:
serie_f_6_m_50 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=50))

In [ ]:
x_range = np.linspace(-np.pi, np.pi, 300)
real = [(f).subs(x, i) for i in x_range]
f6m3 = [serie_f_6_m_3.subs(x, i) for i in x_range]
f6m10 = [serie_f_6_m_10.subs(x, i) for i in x_range]
f6m20 = [serie_f_6_m_20.subs(x, i) for i in x_range]
f6m50 = [serie_f_6_m_50.subs(x, i) for i in x_range]

f6 = [serie_f_6.subs(x, i) for i in x_range]

plt.plot(x_range, real, label="E**x")
plt.plot(x_range, f6, label='f6')
plt.plot(x_range, f6m3, label="f6disc-m3")
plt.plot(x_range, f6m10, label="f6disc-m10")
plt.plot(x_range, f6m20, label="f6disc-m20")
plt.plot(x_range, f6m50, label="f6disc-m50")


plt.legend()

#### EJEMPLO 25 ($f(x) = x$)

In [ ]:
f = x

In [ ]:
coefs_fourier(f, x, [-pi, pi], n_coefs=6)

In [ ]:
serie_f_3 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=3))
serie_f_6 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=6))
serie_f_15 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=15))

In [ ]:
x_range = np.linspace(-np.pi, np.pi, 300)
real = [(f).subs(x, i) for i in x_range]
f3 = [serie_f_3.subs(x, i) for i in x_range]
f6 = [serie_f_6.subs(x, i) for i in x_range]
f15 = [serie_f_15.subs(x, i) for i in x_range]

plt.plot(x_range, real, label="x")
plt.plot(x_range, f3, label='f3')
plt.plot(x_range, f6, label="f6")
plt.plot(x_range, f15, label="f15")

plt.legend()

Ahora hacemos el caso discreto para m=6

In [ ]:
serie_f_6_m_5 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=5))

In [ ]:
serie_f_6_m_10 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=10))

In [ ]:
serie_f_6_m_20 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=20))

In [ ]:
serie_f_6_m_50 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=50))

In [ ]:
x_range = np.linspace(-np.pi, np.pi, 300)
real = [(f).subs(x, i) for i in x_range]
f6m5 = [serie_f_6_m_5.subs(x, i) for i in x_range]
f6m10 = [serie_f_6_m_10.subs(x, i) for i in x_range]
f6m20 = [serie_f_6_m_20.subs(x, i) for i in x_range]
f6m50 = [serie_f_6_m_50.subs(x, i) for i in x_range]

f6 = [serie_f_6.subs(x, i) for i in x_range]

plt.plot(x_range, real, label="x")
plt.plot(x_range, f6, label='f6')
plt.plot(x_range, f6m5, label="f6disc-m3")
plt.plot(x_range, f6m10, label="f6disc-m10")
plt.plot(x_range, f6m20, label="f6disc-m20")
plt.plot(x_range, f6m50, label="f6disc-m50")


plt.legend()

#### EJEMPLO 25 ($f(x) = x^2$)

In [ ]:
f = x ** 2

In [ ]:
coefs_fourier(f, x, [-pi, pi], n_coefs=6)

In [ ]:
N(2*pi**2/3)

In [ ]:
coefs_fourier_discr(f, x, [-np.pi, np.pi], n_coefs=6, m=100)

In [ ]:
serie_f_3 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=3))
serie_f_6 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=6))
serie_f_15 = expand(serie_fourier(f, x, [-pi, pi], n_coefs=15))

In [ ]:
x_range = np.linspace(-np.pi, np.pi, 300)
real = [(f).subs(x, i) for i in x_range]
f3 = [serie_f_3.subs(x, i) for i in x_range]
f6 = [serie_f_6.subs(x, i) for i in x_range]
f15 = [serie_f_15.subs(x, i) for i in x_range]

plt.plot(x_range, real, label="x^2")
plt.plot(x_range, f3, label='f3')
plt.plot(x_range, f6, label="f6")
plt.plot(x_range, f15, label="f15")

plt.legend()

Ahora hacemos el caso discreto para m=6

In [ ]:
serie_f_6_m_3 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=3))

In [ ]:
serie_f_6_m_10 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=10))

In [ ]:
serie_f_6_m_20 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=20))

In [ ]:
serie_f_6_m_50 = expand(serie_fourier(f, x, [-np.pi, np.pi], n_coefs=6, discreto=True, m=50))

In [ ]:
x_range = np.linspace(-np.pi, np.pi, 300)
real = [(f).subs(x, i) for i in x_range]
f6m3 = [serie_f_6_m_3.subs(x, i) for i in x_range]
f6m10 = [serie_f_6_m_10.subs(x, i) for i in x_range]
f6m20 = [serie_f_6_m_20.subs(x, i) for i in x_range]
f6m50 = [serie_f_6_m_50.subs(x, i) for i in x_range]

f6 = [serie_f_6.subs(x, i) for i in x_range]

plt.plot(x_range, real, label="x^2")
plt.plot(x_range, f6, label='f6')
plt.plot(x_range, f6m3, label="f6disc-m3")
plt.plot(x_range, f6m10, label="f6disc-m10")
plt.plot(x_range, f6m20, label="f6disc-m20")
plt.plot(x_range, f6m50, label="f6disc-m50")


plt.legend()

### Aproximación uniforme (Polinomios de Bernstein)
Otro modo de realizar aproximaciones a funciones es empleando la aproximación uniforme. El tma de Weierstrass indica que para $f$ continua en un intervalo cerrado y acotado $I$, para todo $\epsilon > 0$ existe un polinomio $p$ tal que $||f-p||_\infty < \epsilon$. 

Para $I = [0, 1]$, el conjunto de polinomios de grado $n$ que satisface esa condición son los polinomios de Bernstein:
$$B_{n,f}(x) = \sum_{i = 0}^n f\left(\frac{i}{n}\right) {n \choose i} x^i(1-x)^{n-i}$$

Para $I = [a, b]$, hay que realizar el cambio de variable $x = (b-a)t + a$ para $t \in [0,1]$. Con ello, definimos $g(t)$ y tenemos el polinomio
$$B_{n,f}(t) = \sum_{i = 0}^n g\left(\frac{i}{n}\right) {n \choose i} t^i(1-t)^{n-i}$$
Que después reconvertimos a $x$ aplicando el cambio $t = \frac{x - a}{b - a}$

In [ ]:
help(polinomios_bernstein)

#### EJERCICIO 38

In [ ]:
f = abs(x)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=1)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=2)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=3)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=4)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=5)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=6)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=7)

In [ ]:
polinomios_bernstein(f, x, I=[-1, 1], grado=8)

Vemos que los polinomios se repiten por paridad. Esto tiene sentido porque los polinomios solo pueden tener coeficientes pares, ya que abs(x) siempre es positivo.
Vamos a plotear los polinomios hasta un grado alto.

In [ ]:
x_range = np.linspace(-1, 1, 300)
real = [(f).subs(x, i) for i in x_range]
B2 = [polinomios_bernstein(f, x, I=[-1, 1], grado=2).subs(x, i) for i in x_range]
B6 = [polinomios_bernstein(f, x, I=[-1, 1], grado=6).subs(x, i) for i in x_range]
B10 = [polinomios_bernstein(f, x, I=[-1, 1], grado=10).subs(x, i) for i in x_range]
B100 = [polinomios_bernstein(f, x, I=[-1, 1], grado=100).subs(x, i) for i in x_range]

plt.plot(x_range, real, label="abs(x)")
plt.plot(x_range, B2, label='B2')
plt.plot(x_range, B6, label='B6')
plt.plot(x_range, B10, label='B10')
plt.plot(x_range, B100, label='B100')

plt.legend()

Vemos que la curva se acerca cada vez más a abs(x).

#### EJEMPLO 26

In [ ]:
f = E**(3*x) * sin(pi * x)
f

In [ ]:
polinomios_bernstein(f, x, I=[0, 1], grado=1)

In [ ]:
polinomios_bernstein(f, x, I=[0, 1], grado=2)

In [ ]:
polinomios_bernstein(f, x, I=[0, 1], grado=3)

In [ ]:
polinomios_bernstein(f, x, I=[0, 1], grado=4)

Vemos que los polinomios se repiten por paridad. Esto tiene sentido porque los polinomios solo pueden tener coeficientes pares, ya que abs(x) siempre es positivo.
Vamos a plotear los polinomios hasta un grado alto.

In [ ]:
x_range = np.linspace(0, 1, 300)
real = [(f).subs(x, i) for i in x_range]
B2 = [polinomios_bernstein(f, x, I=[0, 1], grado=2).subs(x, i) for i in x_range]
B4 = [polinomios_bernstein(f, x, I=[0, 1], grado=4).subs(x, i) for i in x_range]
B8 = [polinomios_bernstein(f, x, I=[0, 1], grado=8).subs(x, i) for i in x_range]
B16 = [polinomios_bernstein(f, x, I=[0, 1], grado=16).subs(x, i) for i in x_range]
B32 = [polinomios_bernstein(f, x, I=[0, 1], grado=32).subs(x, i) for i in x_range]

plt.plot(x_range, real, label="f")
plt.plot(x_range, B2, label='B2')
plt.plot(x_range, B4, label='B4')
plt.plot(x_range, B8, label='B8')
plt.plot(x_range, B16, label='B16')
plt.plot(x_range, B32, label='B32')

plt.legend()

Ahora vamos a plotear el error entre la función real y la aproximación. Vemos que conforme el polinomio es más complejo, más se acerca a la función, pero también salen artefactos en la computación.

In [ ]:
x_range = np.linspace(0, 1, 300)
real = [(f).subs(x, i) for i in x_range]
B2 = [polinomios_bernstein(f, x, I=[0, 1], grado=2).subs(x, i) for i in x_range]
B4 = [polinomios_bernstein(f, x, I=[0, 1], grado=4).subs(x, i) for i in x_range]
B8 = [polinomios_bernstein(f, x, I=[0, 1], grado=8).subs(x, i) for i in x_range]
B16 = [polinomios_bernstein(f, x, I=[0, 1], grado=16).subs(x, i) for i in x_range]
B32 = [polinomios_bernstein(f, x, I=[0, 1], grado=32).subs(x, i) for i in x_range]

plt.plot(x_range, np.array(real) - np.array(B2), label="2")
plt.plot(x_range, np.array(real) - np.array(B4), label="4")
plt.plot(x_range, np.array(real) - np.array(B8), label="8")
plt.plot(x_range, np.array(real) - np.array(B32), label="32")

plt.legend()